In [7]:
import os
from dotenv import load_dotenv

# Get the path two folders back
env_path = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Load the .env file from the calculated path
dotenv_path = os.path.join(env_path, ".env")
load_dotenv(dotenv_path)

print("Environment Path:", env_path)

Environment Path: /Users/herooooooooo/ML_journey


In [ ]:
from openai import OpenAI

client = OpenAI()

In [15]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[ 
        # {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    max_tokens=50,
    temperature=0.7,
    logprobs=True
)

In [20]:
import math

for token, _, logprob, _ in response.choices[0].logprobs.content:
    print(token, math.exp(logprob[1]))  

('token', 'The') 1.0
('token', ' capital') 1.0
('token', ' of') 1.0
('token', ' France') 1.0
('token', ' is') 1.0
('token', ' Paris') 0.9999980183344259
('token', '.') 0.9999993295729128


In [22]:
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(..., description="The name of the person")
    age: int = Field(..., ge=0, description="The age of the person, must be a non-negative integer")
    job: str = Field(..., description="The job of the person")
    favorite_color: str = Field(..., description="The favorite color of the person")
    

In [23]:
prompt = "Alice is a 30-year-old software engineer who loves blue."

response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[ 
        {"role": "system", "content": "based on input text, fill the data model with all of the information you can find"},
        {"role": "user", "content": prompt}
    ], 
    max_tokens=50,
    temperature=0.7,
    logprobs=True,
    response_format=PersonInfo
)

In [24]:
response.choices[0].message.content

'{"name":"Alice","age":30,"job":"software engineer","favorite_color":"blue"}'

In [25]:
from structured_logprobs import add_logprobs

completion = add_logprobs(response)

In [29]:
# completion.log_probs # convert the logprobs in math.exp format
{k: math.exp(v) for k, v in completion.log_probs[0].items()}

{'name': 1.0, 'age': 1.0, 'job': 0.9241415714110701, 'favorite_color': 1.0}